In [1]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint


from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor
from architectures import vgg16net, smallnet1, smallnet2, smallnet3
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn, pipeline

from architectures import build_net, smallnet3

wandb.login()


wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [2]:
""""
grid search
'learning_rate': {
            'values': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]
"""

'"\ngrid search\n\'learning_rate\': {\n            \'values\': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]\n'

In [22]:
"""config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'epochs':{
            'value': 120 #150
        },
        'lin_layer_size': {
            'values': [100]
        },
        'first_lin_lay':{
            'values':[71744]
        },
        'optimizer': {
            'values': ['adam']
        },
        'scheduler': {
            'values': [0.2]
        },
        'learning_rate': { 'values': [6.53E-05]
        },
        'weight_decay':{
            'values': [2.00E-05]  #1e-5,2e-5, 3e-5,4e-5]
        },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy']
        },
        'data_set':{
            'values':['Augmented']
        },
        'channels': {
            'values': [3]
        },
        'ks':{
            'values':[(3,5)]
        },

        'num_classes': {
            'values': [11]
        },
         'model_name' : {'values': ['smallnet2']},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']}
        },
    }
   
"""
config = dict(
    epochs= 120, 
    learning_rate = 9.07E-05, #5.81E-05, #9.07E-05, #6.53E-05, #8.41E-05, #6e-5, #3.0779e-5,
    dataset= 'IDSW_Aug',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 2.00E-05, #1.00E-05,#2e-5, #4e-5,
    dropout = 0.3, #0.4,
    first_lin_lay = 274688, #15872, #1069888, #71744, #1069888,#7118208, #248832,
    lin_layer_size= 100,
    ks = (3,5),
    in_chan = 3,
    image_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/',
    model_name = 'smallnet2',
    channels = 3,
    num_classes = 11,
    loss_fn = 'CrossEntropy',
    scheduler =0.1 #0.2 #0.01
    
)

col_dict = {
    'colour': 'colour',
    'size': [226,72],
    'padding': 5,
    'model_size': '3c2l'
}

title = f"IDSW_3c2l_e120_{col_dict['size']}_2_300124"
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location': r'pickles/'}
                #r'/its/home/nn268/antvis/optics/

#modelSize_3conv2lin_IDSW_{col_dict['colour']}_{col_dict['size']}_pad_{col_dict['padding']}
#sweep_id = wandb.sweep(config, project=f"3c2l_11336_findbestrandseednum_230124")

In [4]:
print(config)

{'epochs': 120, 'learning_rate': 6.53e-05, 'dataset': 'IDSW_Aug', 'architecture': 'CNN', 'optimizer': 'adam', 'weight_decay': 2e-05, 'dropout': 0.4, 'first_lin_lay': 71744, 'lin_layer_size': 100, 'ks': (3, 5), 'in_chan': 3, 'image_path': '/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/', 'model_name': 'smallnet2', 'channels': 3, 'num_classes': 11, 'loss_fn': 'CrossEntropy', 'scheduler': 0.2}


In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:

#pipeline(config, col_dict, save_dict, title=save_dict['Run'], device=device)

In [23]:
from fns4wandb import train

model = train(device,col_dict, save_dict, config)

"""def tr(config=None, save_dict=save_dict):
    with wandb.init(config=config):
        seed = random.randint(0, 50)
        config = wandb.config
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model, save_dict,x_test, y_test = hp_sweep(config, col_dict, save_dict, device, seed)
        test_predict_list, test_labels, test_accuracy = test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, config.num_classes)
        save_dict['test_predict_list'] = [c.to('cpu') for c in test_predict_list]
        save_dict['test_labels'] = [c.to('cpu') for c in test_labels]
        save_dict['test_accuracy'] = test_accuracy
        save_dict['seed'] = seed
        title = save_dict['Run']
        with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", 'wb+') as f:
            pickle.dump(save_dict, f)

wandb.agent(sweep_id, tr, count=25)"""

  0%| | 0/120 [00:00<?, ?it/s/its/home/nn268/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
100%|█| 120/120 [27:06<00:00,


avg_train_loss,████▇▆▅▅▄▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▂▂▁▁▁▁
avg_val_loss,████▆▆▄▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▂▂▂▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy_%,▁▁▁▁▂▃▄▄▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇▇▇▇▇████
train_correct,▁▁▁▁▂▃▄▄▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇▇▇▇▇████
train_loss,████▇▆▅▅▄▅▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▂▂▁▁▁▁
val_accuracy_%,▂▁▂▂▄▅▆▆▆▇▇▆▇▇█▇▇▇▇▇▇▇▇█▇██████▇▇███▇▇▇█
val_correct,▂▁▂▂▄▅▆▆▆▇▇▆▇▇█▇▇▇▇▇▇▇▇█▇██████▇▇███▇▇▇█
val_loss,████▆▆▄▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▂▂▂▁
avg_train_loss,2.05812
avg_val_loss,2.07081


TEST ACCURACY:  43.49514563106796


'def tr(config=None, save_dict=save_dict):\n    with wandb.init(config=config):\n        seed = random.randint(0, 50)\n        config = wandb.config\n        device = "cuda:0" if torch.cuda.is_available() else "cpu"\n        model, save_dict,x_test, y_test = hp_sweep(config, col_dict, save_dict, device, seed)\n        test_predict_list, test_labels, test_accuracy = test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, config.num_classes)\n        save_dict[\'test_predict_list\'] = [c.to(\'cpu\') for c in test_predict_list]\n        save_dict[\'test_labels\'] = [c.to(\'cpu\') for c in test_labels]\n        save_dict[\'test_accuracy\'] = test_accuracy\n        save_dict[\'seed\'] = seed\n        title = save_dict[\'Run\']\n        with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", \'wb+\') as f:\n            pickle.dump(save_dict, f)\n\nwandb.agent(sweep_id, tr, count=25)'

In [8]:
import os
print(os.getcwd())

/its/home/nn268/antvis/antvis/optics


In [9]:
"""
config = dict(
    epochs= 80, 
    learning_rate =3.0779e-5,
    dataset= 'IDSW',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    dropout = 0.4,
    first_lin_lay =4096,
    lin_layer_size= 100,
    ks = 3,
    in_chan = 3
)



col_dict = {
    'colour': 'colour',
    'size': [32,32]
}"""

"\nconfig = dict(\n    epochs= 80, \n    learning_rate =3.0779e-5,\n    dataset= 'IDSW',\n    architecture ='CNN',\n    optimizer= 'adam',\n    weight_decay= 4e-5,\n    dropout = 0.4,\n    first_lin_lay =4096,\n    lin_layer_size= 100,\n    ks = 3,\n    in_chan = 3\n)\n\n\n\ncol_dict = {\n    'colour': 'colour',\n    'size': [32,32]\n}"